In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

from pt_runner.v1 import DataHandlerPT


In [ ]:
df = pd.read_excel("data.xlsx", index_col="exp")
_X = df.iloc[:, :-3].values
_Y = df.iloc[:, -3:].values
print(_X.shape)
print(_Y.shape)
data_handler = DataHandlerPT(
    _X=_X, _Y=_Y, scalerX=StandardScaler(), scalerY=StandardScaler()
)

In [ ]:
# Define the model
num_features = data_handler._X.shape[1]
num_outputs = data_handler._Y.shape[1]


class MyModel(nn.Module):
    def __init__(self, num_features, num_outputs):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 24)
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, 6)
        self.fc4 = nn.Linear(6, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = MyModel(num_features, num_outputs)

In [ ]:
from torchsummary import summary

input_size = (num_features,)
summary(model, input_size=input_size)

- Type in your command prompt
    - `tensorboard --logdir=src/T01_basic_pytorch/runs`
- Visit http://localhost:6006

In [ ]:
n_epochs = 1000  # number of epochs to run
batch_size = 10  # size of each batch
validation_interval = 10  # Evaluate every 100 epochs
early_stop_patience = 10  # Stop after 10 validation losses without improvement
best_loss = float("inf")
patience_counter = 0

# Initialize Components
writer = SummaryWriter()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=5)
loss_fn = nn.MSELoss()  # mean square error

data_handler.split_and_scale(test_size=0.2, val_size=0.1, random_state=0)
ds_train = data_handler.get_train()
ds_test = data_handler.get_test()
ds_val = data_handler.get_val()
loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False)

for epoch in tqdm(range(n_epochs), desc="Epoch", total=n_epochs):
    # Training Phase
    model.train()
    epoch_train_loss = 0.0

    for X_batch, Y_batch in loader_train:
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch)
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        # Update weights
        optimizer.step()
        epoch_train_loss += loss.item() * X_batch.size(0)

    avg_train_loss = epoch_train_loss / len(loader_train.dataset)
    writer.add_scalar("Loss/train", avg_train_loss, epoch)

    # Validation Phase
    if epoch % validation_interval == 0:
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, Y_val in loader_val:
                Y_pred = model(X_val)
                val_loss += loss_fn(Y_pred, Y_val).item() * X_val.size(0)

        avg_val_loss = val_loss / len(loader_val.dataset)
        writer.add_scalar("Loss/validation", avg_val_loss, epoch)
        scheduler.step(avg_val_loss)

        # Early Stopping
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            patience_counter += 1
            if patience_counter >= early_stop_patience:
                print(f"Early stopping at epoch {epoch}")
                break

# Final Test Evaluation
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
with torch.no_grad():
    X_test, Y_test = ds_test[:]
    test_pred = model(X_test)
    final_loss = loss_fn(test_pred, Y_test)
    print(f"Final test loss: {final_loss:.4f}")

writer.close()